# BYOL-Pytorch  
Pytorch Implementation of BYOL: Bootstrap Your Own Latent: A New Approach to Self-Supervised Learning (https://arxiv.org/abs/2006.07733).   
Major part of Code is inspired from https://github.com/sthalles/PyTorch-BYOL.  
The Code has more appropriate Naming Convention.
# Default Training
* Running the Python File without any changes trains BYOL with **CIFAR10** Dataset.
* All the Parameters are contained in ___Params Object___ in the script.
# Custom Training
* Change the __Dataset Object__.
* Update the Required Parameters in the ___Params Object___.   


## Import Statement

In [33]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader
from torchvision import datasets
from torchvision.transforms import transforms
import torchvision.models
import cv2
import numpy as np
from tqdm import tqdm

In [34]:
np.random.seed(0)
torch.manual_seed(42)

# Augmentation Functions    
Given an image, augmentation is applied to create two different views. Augmentation used over here is very similar to that of [SimCLR](https://arxiv.org/abs/2002.05709).
### GaussianBlur(Class)   
![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/dd16b16869269dba008d19c0969515a1d50b3ae2)
*   **Parameter** = Kernel Size
*   **Output**     = Gaussian Blur Transformed Image   
### Transforms(Function)
*   **Parameter** = Input Dimension of the Image.
*   **Output**     = Composes a torchvision.transforms Object with all the Transformation functions intact.
### MultiViewDataInjector(Class)
*   **Parameter** = Input Image.
*   **Output**    = Applies **Transforms** Function to result two different augmented image.



In [35]:

class GaussianBlur(object):
    """blur a single image on CPU"""

    def __init__(self, kernel_size):
        radias = kernel_size // 2
        kernel_size = radias * 2 + 1
        self.blur_h = nn.Conv2d(3, 3, kernel_size=(kernel_size, 1),
                                stride=1, padding=0, bias=False, groups=3)
        self.blur_v = nn.Conv2d(3, 3, kernel_size=(1, kernel_size),
                                stride=1, padding=0, bias=False, groups=3)
        self.k = kernel_size
        self.r = radias

        self.blur = nn.Sequential(
            nn.ReflectionPad2d(radias),
            self.blur_h,
            self.blur_v
        )

        self.pil_to_tensor = transforms.ToTensor()
        self.tensor_to_pil = transforms.ToPILImage()

    def __call__(self, img):
        img = self.pil_to_tensor(img).unsqueeze(0)

        sigma = np.random.uniform(0.1, 2.0)
        x = np.arange(-self.r, self.r + 1)
        x = np.exp(-np.power(x, 2) / (2 * sigma * sigma))
        x = x / x.sum()
        x = torch.from_numpy(x).view(1, -1).repeat(3, 1)

        self.blur_h.weight.data.copy_(x.view(3, 1, self.k, 1))
        self.blur_v.weight.data.copy_(x.view(3, 1, 1, self.k))

        with torch.no_grad():
            img = self.blur(img)
            img = img.squeeze()

        img = self.tensor_to_pil(img)

        return img

In [36]:
def Transforms(Input_Dim,S=1):
    Color_Jitter = transforms.ColorJitter(0.8*S,0.8*S,0.8*S,0.2*S)
    Data_Transforms = transforms.Compose([transforms.RandomResizedCrop(size=Input_Dim[0]),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.RandomApply([Color_Jitter],p=0.75),
                                         transforms.RandomGrayscale(p=0.2),
                                         GaussianBlur(int(0.1*Input_Dim[0])),
                                         transforms.ToTensor(),
                                        ])
    return Data_Transforms

In [37]:
class MultiViewDataInjector(object):
    def __init__(self,Transforms):
        self.transforms = Transforms
    def __call__(self,Sample,*Consistent_Flip):
        if Consistent_Flip:
            Sample  =  torchvision.transforms.RandomHorizontalFlip()
        Output = [transforms(Sample) for transforms in self.transforms]
        return Output

# Model   
Contains two basic Neural Networks
*  **MLP_Base** - Creates the **Latent Space** from the Encoder.
*  **Skeleton Net** - Encompases MLP_BASE for **Latent Space** creation and uses **ResNet18** to learn Feature Representations.


In [38]:
class MLP_Base(nn.Module):
    def __init__(self,Inp,Hidden,Projection):
        super(MLP_Base,self).__init__()
        self.Linear1 = nn.Linear(Inp,Hidden)
        self.BatchNorm = nn.BatchNorm1d(Hidden)
        self.Linear2 = nn.Linear(Hidden,Projection)
    def forward(self,Input):
        Linear_Inp = torch.relu(self.BatchNorm(self.Linear1(Input)))
        Linear_Out = self.Linear2(Linear_Inp)
        return Linear_Out

In [39]:
class SkeletonNet(nn.Module):
    def __init__(self,Hid,Proj):
        super(SkeletonNet,self).__init__()
        Resnet = torchvision.models.resnet18(pretrained=False)
        self.Encoder = torch.nn.Sequential(*list(Resnet.children())[:-1])
        self.Proj = MLP_Base(Resnet.fc.in_features,Hid,Proj)
    def forward(self,Input):
        Enc_Out = self.Encoder(Input)
        Enc_Out = Enc_Out.view(Enc_Out.size(0),Enc_Out.size(1))
        Final = self.Proj(Enc_Out)
        return Final

# Training Class

In [40]:
class BYOL:
    def __init__(self,Online_Net,Target_Net,Predictor,Optim,Params):
        self.Online_Net = Online_Net
        self.Target_Net = Target_Net
        self.Predictor  = Predictor
        self.Optim      = Optim
        self.Device     = Params['Device']
        self.Epochs     = Params['Epochs']
        self.Moment        = Params['M']
        self.Batch_Size = Params['Batch_Size']
        self.Save_Path = 'Models/BYOL.pth'
    @torch.no_grad()
    def Update_Target_Params(self):
        for Param_Online,Param_Target in zip(self.Online_Net.parameters(),self.Target_Net.parameters()):
            Param_Target = Param_Target.data *self.Moment + Param_Online.data*(1-self.Moment)
    @staticmethod
    def Loss(Rep1,Rep2):
        Norm_Rep1 = F.normalize(Rep1,dim=-1,p=2) #L2-Normalized Rep One
        Norm_Rep2 = F.normalize(Rep2,dim=-1,p=2) #L2 Normalized Rep Two
        Loss = -2 * (Norm_Rep1*Norm_Rep2).sum(dim=-1)
        return Loss
    def Init_Target_Network(self):
        for Param_Online,Param_Target in zip(self.Online_Net.parameters(),self.Target_Net.parameters()):
            Param_Target.data.copy_(Param_Online.data) #Init Target with Param_Online
            Param_Target.requires_grad = False
    def TrainLoop(self,View1,View2):
        self.Optim.zero_grad()
        Pred1 = self.Predictor(self.Online_Net(View1))
        Pred2 = self.Predictor(self.Online_Net(View2))
        with torch.no_grad():
            Target2 = self.Target_Net(View1)
            Target1 = self.Target_Net(View2)
        Loss_Calc = self.Loss(Pred1,Target1) + self.Loss(Pred2,Target2)
        return Loss_Calc.mean()
    def Train(self,Trainset):
        TrainLoader = torch.utils.data.DataLoader(Trainset,batch_size=self.Batch_Size,drop_last=False,shuffle=True)
        self.Init_Target_Network()
        for Epoch in range(self.Epochs):
          Loss_Count = 0.0
          print("Epoch {}".format(Epoch))
          for (View_1,View_2),_ in tqdm(TrainLoader):
              View_1 = View_1.to(self.Device)
              View_2 = View_2.to(self.Device)
              Loss = self.TrainLoop(View_1,View_2)
              Loss_Count += Loss.item()
              Loss.backward()
              self.Optim.step()
              self.Update_Target_Params()
          Epoch_Loss = Loss_Count/len(TrainLoader)
          print("\n Epoch {} Loss:{} : ".format(Epoch,Epoch_Loss))
        os.makedirs(os.path.dirname(self.Save_Path), exist_ok=True)
        self.Save(self.Save_Path)
    def Save(self,Save):
        torch.save({'Online_Net':self.Online_Net.state_dict(),
                    'Enc_Net':self.Online_Net.Encoder.state_dict(),
                    'Target_Net':self.Target_Net.state_dict(),
                    'Optim':self.Optim.state_dict()},Save)


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class BYOL:
    def __init__(self,Online_Net,Target_Net,Predictor,Optim,Params):
        self.Online_Net = Online_Net
        self.Target_Net = Target_Net
        self.Predictor  = Predictor
        self.Optim      = Optim
        self.Device     = Params['Device']
        self.Epochs     = Params['Epochs']
        self.Moment        = Params['M']
        self.Batch_Size = Params['Batch_Size']
        self.Save_Path = 'Models/BYOL.pth'
    @torch.no_grad()
    def Update_Target_Params(self):
        for Param_Online,Param_Target in zip(self.Online_Net.parameters(),self.Target_Net.parameters()):
            Param_Target = Param_Target.data *self.Moment + Param_Online.data*(1-self.Moment)
    @staticmethod
    def Loss(Rep1,Rep2):
        Norm_Rep1 = F.normalize(Rep1,dim=-1,p=2) #L2-Normalized Rep One
        Norm_Rep2 = F.normalize(Rep2,dim=-1,p=2) #L2 Normalized Rep Two
        Loss = -2 * (Norm_Rep1*Norm_Rep2).sum(dim=-1)
        return Loss
    def Init_Target_Network(self):
        for Param_Online,Param_Target in zip(self.Online_Net.parameters(),self.Target_Net.parameters()):
            Param_Target.data.copy_(Param_Online.data) #Init Target with Param_Online
            Param_Target.requires_grad = False
    def TrainLoop(self,View1,View2):
        self.Optim.zero_grad()
        Pred1 = self.Predictor(self.Online_Net(View1))
        Pred2 = self.Predictor(self.Online_Net(View2))
        with torch.no_grad():
            Target2 = self.Target_Net(View1)
            Target1 = self.Target_Net(View2)
        Loss_Calc = self.Loss(Pred1,Target1) + self.Loss(Pred2,Target2)
        return Loss_Calc.mean()


    def Train(self, Trainset, Evalset):
        """Modified Train method to include accuracy tracking."""
        TrainLoader = torch.utils.data.DataLoader(Trainset, batch_size=self.Batch_Size, drop_last=False, shuffle=True)
        self.Init_Target_Network()

        epoch_accuracies = []
        for Epoch in range(self.Epochs):
            Loss_Count = 0.0
            print("Epoch {}".format(Epoch))
            for (View_1, View_2), _ in tqdm(TrainLoader):
                View_1 = View_1.to(self.Device)
                View_2 = View_2.to(self.Device)
                Loss = self.TrainLoop(View_1, View_2)
                Loss_Count += Loss.item()
                self.Optim.zero_grad()
                Loss.backward()
                self.Optim.step()
                self.Update_Target_Params()

            Epoch_Loss = Loss_Count / len(TrainLoader)
            print(f"Epoch {Epoch + 1} Loss: {Epoch_Loss:.4f}")

            # Evaluate after each epoch
            accuracy = self.Evaluate(Evalset)
            epoch_accuracies.append(accuracy)
            print(f"Epoch {Epoch + 1} Accuracy: {accuracy:.2f}%")

        self.Save(self.Save_Path)

        # Plot accuracies
        self.Plot_Accuracies(epoch_accuracies)

    @staticmethod
    def Plot_Accuracies(accuracies):
        """Plot the accuracies after training."""
        plt.figure(figsize=(10, 6))
        plt.plot(range(1, len(accuracies) + 1), accuracies, marker='o', linestyle='-', color='b')
        plt.title('Accuracy Over Epochs', fontsize=14)
        plt.xlabel('Epochs', fontsize=12)
        plt.ylabel('Accuracy (%)', fontsize=12)
        plt.grid()
        plt.show()


    def Save(self,Save):
        torch.save({'Online_Net':self.Online_Net.state_dict(),
                    'Enc_Net':self.Online_Net.Encoder.state_dict(),
                    'Target_Net':self.Target_Net.state_dict(),
                    'Optim':self.Optim.state_dict()},Save)

    def Evaluate(self, Evalset):
        """Evaluate the learned representations with a linear classifier."""
        self.Online_Net.eval()  # Put the network in evaluation mode
        Linear_Eval_Loader = torch.utils.data.DataLoader(Evalset, batch_size=self.Batch_Size, shuffle=False)

        #classifier = nn.Linear(self.Online_Net.Encoder[-2].out_features, 10).to(self.Device)  # 10 classes for CIFAR-10
        classifier = nn.Linear(512, 10).to(self.Device)  # 10 classes for CIFAR-10
        nn.init.xavier_uniform_(classifier.weight)
        optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
        criterion = nn.CrossEntropyLoss()

        # Freeze the encoder
        for param in self.Online_Net.parameters():
            param.requires_grad = False

        epochs = 20  # Train the linear classifier for 10 epochs
        classifier.train()
        for epoch in range(epochs):
            total_loss = 0
            correct = 0
            total = 0
            for images, labels in Linear_Eval_Loader:
                images, labels = images.to(self.Device), labels.to(self.Device)
                with torch.no_grad():
                    features = self.Online_Net.Encoder(images).squeeze()  # Extract features from the encoder
                outputs = classifier(features)
                loss = criterion(outputs, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, predicted = outputs.max(1)
                correct += predicted.eq(labels).sum().item()
                total += labels.size(0)

            accuracy = 100 * correct / total
            print(f"Linear Evaluation - Epoch [{epoch + 1}/{epochs}], Loss: {total_loss:.4f}, Accuracy: {accuracy:.2f}%")

        return accuracy

# Main Training

In [43]:
Parameters = {'Epochs':200,'M':0.99,'Batch_Size':256,'Device':'cuda','Hidden':512,'Proj':128,'LR':0.03}

In [44]:
Data_Transforms = Transforms((3,32,32))
Dataset = datasets.CIFAR10('./data',download=True,transform=MultiViewDataInjector([Data_Transforms,Data_Transforms]))

Files already downloaded and verified


In [45]:
Online_Network = SkeletonNet(Parameters['Hidden'],Parameters['Proj'])
Predictor = MLP_Base(Online_Network.Proj.Linear2.out_features,Parameters['Hidden'],Parameters['Proj'])
Target_Network = SkeletonNet(Parameters['Hidden'],Parameters['Proj'])

In [46]:
Online_Network.to(Parameters['Device'])
Predictor.to(Parameters['Device'])
Target_Network.to(Parameters['Device'])
print("Models Made.")

Models Made.


In [47]:
Optimizer = torch.optim.SGD(list(Online_Network.parameters())+list(Predictor.parameters()),lr=0.03)

In [48]:
Trainer = BYOL(Online_Network,Target_Network,Predictor,Optimizer,Parameters)

In [49]:
Trainer.Train(Dataset)

Epoch 0


100%|██████████| 196/196 [00:45<00:00,  4.34it/s]



 Epoch 0 Loss:-2.1758900406789414 : 
Epoch 1


100%|██████████| 196/196 [00:46<00:00,  4.21it/s]



 Epoch 1 Loss:-2.2591226271220615 : 
Epoch 2


...


 Epoch 197 Loss:-2.275902263972224 : 
Epoch 198


100%|██████████| 196/196 [00:51<00:00,  3.78it/s]



 Epoch 198 Loss:-2.275442916519788 : 
Epoch 199


100%|██████████| 196/196 [00:52<00:00,  3.74it/s]


 Epoch 199 Loss:-2.277152805912251 : 


In [ ]:
Trainer.Save('Models/BYOL.pth')